# Imports

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from lxml import html
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs

import pprint

from openpyxl import load_workbook

from scipy.stats import truncnorm

from datetime import datetime

# Get data from retrosheet

In [4]:
session_a = requests.session()

In [5]:
url_a = 'https://www.retrosheet.org/boxesetc/1971/Y_1971.htm'

In [6]:
a = session_a.get(url_a)

In [7]:
soup_a = BeautifulSoup(a.text, 'html.parser')
soup_a

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0//EN" "https://www.w3.org/TR/REC-html40/strict.dtd">

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Y_descr.htm">Read Me</a></pre>
<head>
<title>The 1971 Season</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a></li>
<li><a href="#">About ↓</a>
<ul>
<li><a href="https://www.retrosheet.org/site.htm">Overview</a></li>
<li><a href="https://www.retrosheet.org/faq.htm">FAQ</a></li>
</ul>
</li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players</a></li>
<l

In [9]:
tags = soup_a.find_all('a', {'href': re.compile(r'../1971/T\w.*')})
tags


[<a href="../1971/TPIT01971.htm">Pittsburgh Pirates</a>,
 <a href="../1971/TSLN01971.htm">St. Louis Cardinals</a>,
 <a href="../1971/TCHN01971.htm">Chicago Cubs</a>,
 <a href="../1971/TNYN01971.htm">New York Mets</a>,
 <a href="../1971/TMON01971.htm">Montreal Expos</a>,
 <a href="../1971/TPHI01971.htm">Philadelphia Phillies</a>,
 <a href="../1971/TSFN01971.htm">San Francisco Giants</a>,
 <a href="../1971/TLAN01971.htm">Los Angeles Dodgers</a>,
 <a href="../1971/TATL01971.htm">Atlanta Braves</a>,
 <a href="../1971/TCIN01971.htm">Cincinnati Reds</a>,
 <a href="../1971/THOU01971.htm">Houston Astros</a>,
 <a href="../1971/TSDN01971.htm">San Diego Padres</a>,
 <a href="../1971/TBAL01971.htm">Baltimore Orioles</a>,
 <a href="../1971/TDET01971.htm">Detroit Tigers</a>,
 <a href="../1971/TBOS01971.htm">Boston Red Sox</a>,
 <a href="../1971/TNYA01971.htm">New York Yankees</a>,
 <a href="../1971/TWS201971.htm">Washington Senators</a>,
 <a href="../1971/TCLE01971.htm">Cleveland Indians</a>,
 <a hr

In [10]:
my_hrefs = []
for t in tags:
    link = t.get('href')
    p = urlparse(link).path.rsplit("/", 1)[-1]
    my_hrefs.append(p)
my_hrefs

['TPIT01971.htm',
 'TSLN01971.htm',
 'TCHN01971.htm',
 'TNYN01971.htm',
 'TMON01971.htm',
 'TPHI01971.htm',
 'TSFN01971.htm',
 'TLAN01971.htm',
 'TATL01971.htm',
 'TCIN01971.htm',
 'THOU01971.htm',
 'TSDN01971.htm',
 'TBAL01971.htm',
 'TDET01971.htm',
 'TBOS01971.htm',
 'TNYA01971.htm',
 'TWS201971.htm',
 'TCLE01971.htm',
 'TOAK01971.htm',
 'TKCA01971.htm',
 'TCHA01971.htm',
 'TCAL01971.htm',
 'TMIN01971.htm',
 'TMIL01971.htm']

In [12]:
my_teams = [char[1:4] for char in my_hrefs]

In [13]:
session = requests.session()

In [14]:
url = "https://www.retrosheet.org/boxesetc/1971/TPIT01971.htm"

In [15]:
s = session.get(url)

In [16]:
soup = BeautifulSoup(s.text, 'html.parser')

In [17]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0//EN" "https://www.w3.org/TR/REC-html40/strict.dtd">

<html dir="LTR" lang="EN">
<pre><a href="../MISC/Tdescr.htm">Read Me</a></pre>
<head>
<title>The 1971 Pittsburgh Pirates</title>
<link href="https://www.retrosheet.org/menubar/menubar.css" rel="stylesheet" type="text/css"/>
<script src="https://www.retrosheet.org/menubar/menubar.js" type="text/javascript"></script>
</head>
<body>
<p class="nopad"><img alt="Retrosheet" class="bancenter" height="46" src="https://www.retrosheet.org/menubar/retro-logo.gif" width="400"/></p>
<div class="mbcenter">
<ul class="nav">
<li><a href="https://www.retrosheet.org/">Home</a></li>
<li><a href="#">About ↓</a>
<ul>
<li><a href="https://www.retrosheet.org/site.htm">Overview</a></li>
<li><a href="https://www.retrosheet.org/faq.htm">FAQ</a></li>
</ul>
</li>
<li><a href="#">Games/People/Parks ↓</a>
<ul>
<li><a href="#">People →</a>
<ul>
<li><a href="https://www.retrosheet.org/boxesetc/index.html#Players">Players<

In [18]:
first_pre = soup.find('h4')
first_pre

<h4>Home and Away Record</h4>

In [19]:
first_pre.find_next_siblings('pre')[0].text

'                   Total                      Home                       Away\nTeam       G    W    L   RS   RA      G    W    L   RS   RA      G    W    L   RS   RA\nTotal    162   97   65  788  599     80   52   28  393  279     82   45   37  395  320\nATL N     12    8    4   83   52      5    4    1   33   14      7    4    3   50   38\nCHI N     18   12    6   95   48      9    7    2   55   19      9    5    4   40   29\nCIN N     12    7    5   53   40      6    5    1   32   15      6    2    4   21   25\nHOU N     12    8    4   54   43      6    4    2   29   25      6    4    2   25   18\nLA  N     12    8    4   63   50      6    3    3   27   27      6    5    1   36   23\nMON N     18   11    7   89   73      9    7    2   52   43      9    4    5   37   30\nNY  N     18    8   10   45   50      9    4    5   31   32      9    4    5   14   18\nPHI N     18   12    6  104   80      9    6    3   40   29      9    6    3   64   51\nSD  N     12    9    3   50   27      6 

In [20]:
data = first_pre.find_next_siblings('pre')[0].text
data

'                   Total                      Home                       Away\nTeam       G    W    L   RS   RA      G    W    L   RS   RA      G    W    L   RS   RA\nTotal    162   97   65  788  599     80   52   28  393  279     82   45   37  395  320\nATL N     12    8    4   83   52      5    4    1   33   14      7    4    3   50   38\nCHI N     18   12    6   95   48      9    7    2   55   19      9    5    4   40   29\nCIN N     12    7    5   53   40      6    5    1   32   15      6    2    4   21   25\nHOU N     12    8    4   54   43      6    4    2   29   25      6    4    2   25   18\nLA  N     12    8    4   63   50      6    3    3   27   27      6    5    1   36   23\nMON N     18   11    7   89   73      9    7    2   52   43      9    4    5   37   30\nNY  N     18    8   10   45   50      9    4    5   31   32      9    4    5   14   18\nPHI N     18   12    6  104   80      9    6    3   40   29      9    6    3   64   51\nSD  N     12    9    3   50   27      6 

In [21]:
table = pd.DataFrame([x.strip('\t') for x in data.split('\n')], columns=["unsplit"])
table.drop(table.index[0:3], axis=0, inplace=True)
table.drop(table.index[-1], axis=0, inplace=True)
table = table.reset_index()
table = table.drop(columns="index")
table

,unsplit
0,ATL N 12 8 4 83 52 5 4 ...
1,CHI N 18 12 6 95 48 9 7 ...
2,CIN N 12 7 5 53 40 6 5 ...
3,HOU N 12 8 4 54 43 6 4 ...
4,LA N 12 8 4 63 50 6 3 ...
5,MON N 18 11 7 89 73 9 7 ...
6,NY N 18 8 10 45 50 9 4 ...
7,PHI N 18 12 6 104 80 9 6 ...
8,SD N 12 9 3 50 27 6 6 ...
9,SF N 12 3 9 62 72 6 2 ...


In [22]:
table.shape

(11, 1)

In [23]:
my_cols = [
    'opponent', 'league', 'total_g', 'total_w', 'total_l', 'total_rs', 'total_ra', 
    'home_g', 'home_w', 'home_l', 'home_rs', 'home_ra', 
    'away_g', 'away_w', 'away_l', 'away_rs', 'away_ra'
]
df = pd.DataFrame() 
df[my_cols] = table['unsplit'].str.split('\s+',expand=True)
df["team"] = "PIT"
df

,opponent,league,total_g,total_w,total_l,total_rs,total_ra,home_g,home_w,home_l,home_rs,home_ra,away_g,away_w,away_l,away_rs,away_ra,team
0,ATL,N,12,8,4,83,52,5,4,1,33,14,7,4,3,50,38,PIT
1,CHI,N,18,12,6,95,48,9,7,2,55,19,9,5,4,40,29,PIT
2,CIN,N,12,7,5,53,40,6,5,1,32,15,6,2,4,21,25,PIT
3,HOU,N,12,8,4,54,43,6,4,2,29,25,6,4,2,25,18,PIT
4,LA,N,12,8,4,63,50,6,3,3,27,27,6,5,1,36,23,PIT
5,MON,N,18,11,7,89,73,9,7,2,52,43,9,4,5,37,30,PIT
6,NY,N,18,8,10,45,50,9,4,5,31,32,9,4,5,14,18,PIT
7,PHI,N,18,12,6,104,80,9,6,3,40,29,9,6,3,64,51,PIT
8,SD,N,12,9,3,50,27,6,6,0,30,14,6,3,3,20,13,PIT
9,SF,N,12,3,9,62,72,6,2,4,28,29,6,1,5,34,43,PIT


In [24]:
type_dict = {
    'total_g': 'int', 
    'total_w': 'int',
    'total_l': 'int',
    'total_rs': 'int',
    'total_ra': 'int',
    'home_g': 'int', 
    'home_w': 'int',
    'home_l': 'int',
    'home_rs': 'int',
    'home_ra': 'int',
    'away_g': 'int', 
    'away_w': 'int',
    'away_l': 'int',
    'away_rs': 'int',
    'away_ra': 'int',
}
df = df.astype(type_dict)
df

,opponent,league,total_g,total_w,total_l,total_rs,total_ra,home_g,home_w,home_l,home_rs,home_ra,away_g,away_w,away_l,away_rs,away_ra,team
0,ATL,N,12,8,4,83,52,5,4,1,33,14,7,4,3,50,38,PIT
1,CHI,N,18,12,6,95,48,9,7,2,55,19,9,5,4,40,29,PIT
2,CIN,N,12,7,5,53,40,6,5,1,32,15,6,2,4,21,25,PIT
3,HOU,N,12,8,4,54,43,6,4,2,29,25,6,4,2,25,18,PIT
4,LA,N,12,8,4,63,50,6,3,3,27,27,6,5,1,36,23,PIT
5,MON,N,18,11,7,89,73,9,7,2,52,43,9,4,5,37,30,PIT
6,NY,N,18,8,10,45,50,9,4,5,31,32,9,4,5,14,18,PIT
7,PHI,N,18,12,6,104,80,9,6,3,40,29,9,6,3,64,51,PIT
8,SD,N,12,9,3,50,27,6,6,0,30,14,6,3,3,20,13,PIT
9,SF,N,12,3,9,62,72,6,2,4,28,29,6,1,5,34,43,PIT


In [25]:
df["total_win_pct"] = (df["total_w"] / df["total_g"]).astype(float)
df

,opponent,league,total_g,total_w,total_l,total_rs,total_ra,home_g,home_w,home_l,home_rs,home_ra,away_g,away_w,away_l,away_rs,away_ra,team,total_win_pct
0,ATL,N,12,8,4,83,52,5,4,1,33,14,7,4,3,50,38,PIT,0.666667
1,CHI,N,18,12,6,95,48,9,7,2,55,19,9,5,4,40,29,PIT,0.666667
2,CIN,N,12,7,5,53,40,6,5,1,32,15,6,2,4,21,25,PIT,0.583333
3,HOU,N,12,8,4,54,43,6,4,2,29,25,6,4,2,25,18,PIT,0.666667
4,LA,N,12,8,4,63,50,6,3,3,27,27,6,5,1,36,23,PIT,0.666667
5,MON,N,18,11,7,89,73,9,7,2,52,43,9,4,5,37,30,PIT,0.611111
6,NY,N,18,8,10,45,50,9,4,5,31,32,9,4,5,14,18,PIT,0.444444
7,PHI,N,18,12,6,104,80,9,6,3,40,29,9,6,3,64,51,PIT,0.666667
8,SD,N,12,9,3,50,27,6,6,0,30,14,6,3,3,20,13,PIT,0.750000
9,SF,N,12,3,9,62,72,6,2,4,28,29,6,1,5,34,43,PIT,0.250000


In [26]:
df2 = df[["team", "opponent", "league", "total_win_pct"]]
df2

,team,opponent,league,total_win_pct
0,PIT,ATL,N,0.666667
1,PIT,CHI,N,0.666667
2,PIT,CIN,N,0.583333
3,PIT,HOU,N,0.666667
4,PIT,LA,N,0.666667
5,PIT,MON,N,0.611111
6,PIT,NY,N,0.444444
7,PIT,PHI,N,0.666667
8,PIT,SD,N,0.750000
9,PIT,SF,N,0.250000


In [27]:
my_hrefs[0]

'TPIT01971.htm'

In [28]:
# Create function to scrape all relevant pages
base_url = "https://www.retrosheet.org/boxesetc/"



def get_head2head(year, hrefs, cols, teams, type_dict):
    session = requests.session()
    events = pd.DataFrame()
    div_count = 0
    for href in hrefs:
       
        result = session.get(base_url + str(year) + "/" + hrefs[div_count])
        
        soup = BeautifulSoup(result.text, 'html.parser')
        
        first_pre = soup.find('h4')
        data = first_pre.find_next_siblings('pre')[0].text
        
        table = pd.DataFrame([x.strip('\t') for x in data.split('\n')], columns=["unsplit"])
        table.drop(table.index[0:3], axis=0, inplace=True)
        table.drop(table.index[-1], axis=0, inplace=True)
        table = table.reset_index()
        table = table.drop(columns="index")
        

        event = pd.DataFrame()
        event[cols] = table['unsplit'].str.split('\s+',expand=True)
        event["team"] = teams[div_count]
        event = event.astype(type_dict)
        event["total_win_pct"] = (event["total_w"] / event["total_g"]).astype(float)
        
        event.loc[(event["total_win_pct"].between(0, 0.02778)), "roll"] = 11
        event.loc[(event["total_win_pct"].between(0.027778, 0.055556)), "roll"] = 12
        event.loc[(event["total_win_pct"].between(0.055556, 0.083333)), "roll"] = 13
        event.loc[(event["total_win_pct"].between(0.083333, 0.111111)), "roll"] = 14
        event.loc[(event["total_win_pct"].between(0.111111, 0.138889)), "roll"] = 15
        event.loc[(event["total_win_pct"].between(0.138889, 0.166667)), "roll"] = 16
        event.loc[(event["total_win_pct"].between(0.166667, 0.194444)), "roll"] = 21
        event.loc[(event["total_win_pct"].between(0.194444, 0.222222)), "roll"] = 22
        event.loc[(event["total_win_pct"].between(0.222222, 0.25)), "roll"] = 23
        event.loc[(event["total_win_pct"].between(0.25, 0.277778)), "roll"] = 24
        event.loc[(event["total_win_pct"].between(0.277778, 0.305556)), "roll"] = 25
        event.loc[(event["total_win_pct"].between(0.305556, 0.333333)), "roll"] = 26
        event.loc[(event["total_win_pct"].between(0.333333, 0.361111)), "roll"] = 31
        event.loc[(event["total_win_pct"].between(0.361111, 0.388889)), "roll"] = 32
        event.loc[(event["total_win_pct"].between(0.388889, 0.416667)), "roll"] = 33
        event.loc[(event["total_win_pct"].between(0.416667, 0.444444)), "roll"] = 34
        event.loc[(event["total_win_pct"].between(0.444444, 0.472222)), "roll"] = 35
        event.loc[(event["total_win_pct"].between(0.472222, 0.5)), "roll"] = 36
        event.loc[(event["total_win_pct"].between(0.50, 0.527778)), "roll"] = 41
        event.loc[(event["total_win_pct"].between(0.527777, 0.555556)), "roll"] = 42
        event.loc[(event["total_win_pct"].between(0.555556, 0.583333)), "roll"] = 43
        event.loc[(event["total_win_pct"].between(0.583333, 0.611111)), "roll"] = 44
        event.loc[(event["total_win_pct"].between(0.611111, 0.638889)), "roll"] = 45
        event.loc[(event["total_win_pct"].between(0.638889, 0.666667)), "roll"] = 46
        event.loc[(event["total_win_pct"].between(0.666667, 0.694444)), "roll"] = 51
        event.loc[(event["total_win_pct"].between(0.694444, 0.722222)), "roll"] = 52
        event.loc[(event["total_win_pct"].between(0.722222, 0.75)), "roll"] = 53
        event.loc[(event["total_win_pct"].between(0.750000, 0.777778)), "roll"] = 54
        event.loc[(event["total_win_pct"].between(0.777778, 0.805556)), "roll"] = 55
        event.loc[(event["total_win_pct"].between(0.805556, 0.833333)), "roll"] = 56
        event.loc[(event["total_win_pct"].between(0.833333, 0.861111)), "roll"] = 61
        event.loc[(event["total_win_pct"].between(0.861111, 0.888889)), "roll"] = 62
        event.loc[(event["total_win_pct"].between(0.888889, 0.916667)), "roll"] = 63
        event.loc[(event["total_win_pct"].between(0.916667, 0.944444)), "roll"] = 64
        event.loc[(event["total_win_pct"].between(0.944444, 0.972222)), "roll"] = 65
        event.loc[(event["total_win_pct"].between(0.972222, 1)), "roll"] = 66
        event["roll"] = event["roll"].replace(np.nan, "")
        
        event2 = event[["team", "opponent", "league", "total_win_pct", "roll"]]
        events = events.append(event2, ignore_index=True)
        
        div_count += 1
        
    return events


In [29]:
pd.set_option('display.max_rows', 300)
win_pct_1971 = get_head2head(1971, my_hrefs, my_cols, my_teams, type_dict)
win_pct_1971

,team,opponent,league,total_win_pct,roll
0,PIT,ATL,N,0.666667,46.0
1,PIT,CHI,N,0.666667,46.0
2,PIT,CIN,N,0.583333,44.0
3,PIT,HOU,N,0.666667,46.0
4,PIT,LA,N,0.666667,46.0
5,PIT,MON,N,0.611111,45.0
6,PIT,NY,N,0.444444,35.0
7,PIT,PHI,N,0.666667,46.0
8,PIT,SD,N,0.750000,54.0
9,PIT,SF,N,0.250000,24.0


In [30]:
table_1971 = pd.pivot_table(win_pct_1971, values='roll', index=['team'],
                    columns=['opponent', 'league'], aggfunc=np.sum, fill_value="")
table_1971

opponent ATL BAL BOS CAL CHI     CIN CLE DET HOU  ... MON  NY     OAK PHI PIT  \
league     N   A   A   A   A   N   N   A   A   N  ...   N   A   N   A   N   N   
team                                              ...                           
ATL                           33  41          41  ...  44      44      46  31   
BAL               41  44  46          53  35      ...      45      45           
BOS           41      41  61          45  46      ...      32      24           
CAL           33  41      35          46  41      ...      41      32           
CHA           31  16  42              24  44      ...      33      45           
CHN       44                      41          33  ...  35      45      45  31   
CIN       41                  41              24  ...  44      46      33  33   
CLE           24  32  31  54              31      ...      35      31           
DET           42  31  41  33          46          ...      42      31           
HOU       41                  44  53              ...  31      33      46  31   
KCA           35  63  42  41          61  31      ...      33      24           
LAN       41                  31  45          42  ...  46      33      44  31   
MIL           24  41  46  32          46  16      ...      16      16           
MIN           16  31  31  45          46  41      ...      46      35           
MON       33                  42  33          46  ...          41      31  32   
NYA           32  45  41  44          42  35      ...              33           
NYN       33                  32  31          44  ...  41              53  42   
OAK           32  54  45  32          46  46      ...      44                   
PHI       31                  32  44          31  ...  46      24          31   
PIT       46                  46  44          46  ...  45      35      46       
SDN       32                  24  35          35  ...  35      33      46  24   
SFN       45                  54  41          41  ...  33      46      41  54   
SLN       41                  41  31          61  ...  53      35      45  32   
WS2           21  31  46  16          45  23      ...      45      24           

opponent  SD  SF STL WAS  
league     N   N   N   A  
team                      
ATL       45  32  41      
BAL                   56  
BOS                   46  
CAL                   31  
CHA                   61  
CHN       54  24  41      
CIN       42  41  46      
CLE                   32  
DET                   54  
HOU       42  41  16      
KCA                   54  
LAN       53  46  41      
MIL                   41  
MIN                   35  
MON       42  44  22      
NYA                   32  
NYN       44  31  42      
OAK                   54  
PHI       31  41  32      
PIT       54  24  45      
SDN           24  31      
SFN       53      33      
SLN       46  44          
WS2                       

[24 rows x 24 columns]

In [31]:
table_1971.to_excel('../data/Fast Action Chart 1971.xlsx', engine='openpyxl')

In [271]:
win_pct_1971["pct"] = round(win_pct_1971["total_win_pct"], 2) * 100

In [274]:
prob_1971 = pd.pivot_table(win_pct_1971, values='pct', index=['team'],
                    columns=['opponent', 'league'], aggfunc=np.sum, fill_value="")
prob_1971

opponent ATL BAL BOS CAL CHI     CIN CLE DET HOU  ... MON  NY     OAK PHI PIT  \
league     N   A   A   A   A   N   N   A   A   N  ...   N   A   N   A   N   N   
team                                              ...                           
ATL                           42  50          50  ...  58      58      67  33   
BAL               50  58  67          72  44      ...      61      64           
BOS           50      50  83          61  67      ...      39      25           
CAL           42  50      44          67  50      ...      50      39           
CHA           33  17  56              25  58      ...      42      61           
CHN       58                      50          42  ...  44      61      61  33   
CIN       50                  50              28  ...  58      67      42  42   
CLE           28  39  33  75              33      ...      44      33           
DET           56  33  50  42          67          ...      56      33           
HOU       50                  58  72              ...  33      42      67  33   
KCA           45  92  56  50          83  33      ...      42      28           
LAN       50                  33  61          56  ...  67      42      58  33   
MIL           25  50  67  39          67  17      ...      17      17           
MIN           17  33  33  61          67  50      ...      67      44           
MON       42                  56  42          67  ...          50      33  39   
NYA           39  61  50  58          56  44      ...              42           
NYN       42                  39  33          58  ...  50              72  56   
OAK           36  75  61  39          67  67      ...      58                   
PHI       33                  39  58          33  ...  67      28          33   
PIT       67                  67  58          67  ...  61      44      67       
SDN       39                  25  44          44  ...  45      42      67  25   
SFN       61                  75  50          50  ...  42      67      50  75   
SLN       50                  50  33          83  ...  74      44      61  39   
WS2           19  33  67  17          61  22      ...      61      25           

opponent  SD  SF STL WAS  
league     N   N   N   A  
team                      
ATL       61  39  50      
BAL                   81  
BOS                   67  
CAL                   33  
CHA                   83  
CHN       75  25  50      
CIN       56  50  67      
CLE                   39  
DET                   78  
HOU       56  50  17      
KCA                   75  
LAN       72  67  50      
MIL                   50  
MIN                   45  
MON       55  58  21      
NYA                   39  
NYN       58  33  56      
OAK                   75  
PHI       33  50  39      
PIT       75  25  61      
SDN           28  33      
SFN       72      42      
SLN       67  58          
WS2                       

[24 rows x 24 columns]

In [276]:
prob_1971.to_excel('../data/Fast Action Chart w Probs 1971.xlsx', engine='openpyxl')

# Generate random score

In [296]:
win_runs = max(1, np.random.normal(5.4, 3, 1).round(0))
lose_runs = max(0, np.random.normal(2.4, 2, 1).round(0))

print(f'Winner score: {win_runs}')
print(f'Loser score: {lose_runs}')

Winner score: [12.]
Loser score: [1.]
